In [53]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from random import seed, shuffle
import os
from datetime import datetime

from pyannote.audio import Pipeline
from pyannote.core import json

from matplotlib import pyplot as plt, lines
import seaborn as sns

from sklearn.cluster import KMeans, SpectralClustering
from sklearn.preprocessing import StandardScaler

from scipy import stats
from scipy.spatial.distance import directed_hausdorff, euclidean, cosine, pdist

from src import (
    downsample as downsamp_audio, 
    embedding_extractor as ee, 
    dimension_reducer as dr, 
    distance_metrics as dm, 
    downsample as ds,
    emb_manipulator as em
)

from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
##Downsample audio to 16kHz
root_dir= "/Users/rahulbrito/Documents/projects/infantvoice/data/Full_Readings/gasser_readings"
audio_files = os.path.join(root_dir,"preprocessed_audios_dur3sec")

#make a folder with today's date for the downsampled audio
down_sample_dir = os.path.join(root_dir,"preprocessed_audios_dur3sec")

#if not os.path.exists(down_sample_dir):
#    os.mkdir(down_sample_dir)

In [ ]:
ds.downsamp_audio(audio_files, down_sample_dir)

processing023.wav


In [28]:
##Diarize audio to find when mother is speaking and visualize it to choose label

diar_dir = os.path.join(root_dir,'diarized')
  
if not os.path.exists(diar_dir):
    os.mkdir(diar_dir)

In [ ]:
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization")

for file in os.listdir(down_sample_dir):
    if file.endswith('wav'):
        diar = pipeline(os.path.join(down_sample_dir,file))
        json.dump_to(diar, os.path.join(diar_dir, os.path.splitext(file)[0]+'.json'))

In [8]:
# Based on output of diarization, manually make speaker map to set which speaker we want
speaker_map = {'021':'SPEAKER_01', '022':'SPEAKER_00', 'BB003':'SPEAKER_00'}

diarized = {os.path.splitext(file)[0]:
            json.load_from(os.path.join(diar_dir,file)).label_timeline(
                speaker_map[os.path.splitext(file)[0]]) 
            for file in os.listdir(diar_dir)}

In [42]:
emb_dir = '/Users/rahulbrito/Documents/projects/infantvoice/data/embeddings'
save_name = datetime.now().strftime('%Y_%m_%d') + '_spont_script_emb.csv'
diarized = None

import glob
all_files = glob.glob(down_sample_dir+'/*/*', recursive=True)
all_files = [file for file in all_files if 'script' in file]

In [ ]:
pyannote_emb = ee.pyannote_extract_directory(all_files, diarized,emb_dir,save_name,window_type="whole",save=True)

Processing/Users/rahulbrito/Documents/projects/infantvoice/data/Full_Readings/gasser_readings/preprocessed_audios_dur3sec/068/068_F_LPP_script_006.wav


In [62]:
len(all_files)

2912

In [61]:
emb = pd.read_csv(os.path.join(emb_dir,save_name), index_col=0)
emb.shape

(2912, 513)

In [63]:
emb

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,part_id
0,-17.866596,-36.472736,2.810474,-13.091801,-29.057281,100.864090,11.403651,2.002557,49.679688,67.245320,...,6.802216,-18.774551,-12.680163,11.479803,-74.303535,-38.301666,-31.930567,30.708248,59.096687,132_M_LPP_script_019
1,-24.720220,-53.471107,-4.527231,-57.362072,-20.035112,33.772250,33.217545,40.182133,-4.187855,45.168850,...,36.793278,39.209064,24.457136,-10.986446,-82.861800,-12.024499,-54.869340,6.032272,10.194699,132_M_LPP_script_025
2,-18.940390,-68.283660,24.777010,16.191492,-5.806833,79.381966,-14.838532,-43.664764,33.277794,-4.101321,...,89.930900,-0.485718,38.250103,-22.176706,-85.565310,-3.649171,-80.308120,69.246400,18.234650,132_M_NWS_script_003
3,-50.842830,7.999563,11.116053,26.489422,-64.757515,-0.371823,-57.951153,11.694829,31.615423,-22.009445,...,-18.523884,-30.315569,9.582836,-38.046140,-106.705040,-84.855484,-30.208515,-19.814474,-63.779877,132_M_HT2_script_007
4,-62.087967,-62.960903,-16.573288,17.406431,47.291813,120.659120,-36.949340,84.342910,-6.193437,37.936527,...,50.988308,27.801933,14.716720,7.936591,-97.921010,-62.073650,-3.162972,-30.842146,85.399254,132_M_HT2_script_013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2907,22.070013,-21.654894,-6.510378,38.028603,12.310236,-23.514390,21.443636,-24.176996,41.468163,-23.341417,...,56.140050,16.357054,65.883030,27.157290,12.287617,2.091420,-36.041145,40.974174,-69.664870,060_F_DHR_script_000
2908,79.328130,-24.792784,73.678760,50.283380,48.158634,-1.352210,13.894097,-54.340030,-13.808758,-59.024372,...,-37.963710,6.424862,13.421726,-41.007927,23.795456,-40.866688,21.520609,77.742130,-38.829280,060_F_DHR_script_014
2909,17.487630,-36.431454,25.003027,32.365345,71.224236,-35.751244,5.099931,-93.188250,-17.091503,-28.857748,...,-6.171011,19.137825,-7.164855,5.279831,36.133120,-65.087890,-0.119039,8.802391,-31.399887,060_F_DHR_script_015
2910,-2.319214,24.743206,40.594780,48.235428,117.881720,-47.165413,50.769550,-94.403786,-12.733907,-100.834000,...,47.530470,54.401325,32.793373,6.507592,28.827675,-72.159450,-82.842740,44.340904,-66.734660,060_F_DHR_script_001
